In [25]:
import pandas as pd
import numpy as np

In [63]:
df = pd.read_csv('../data/mvp_data.csv', parse_dates=True, index_col=0)
tickers = ['EUR', 'GOLD', 'Bitcoin', 'Apple', 'Exxon', 'VISA', 'Oil']
df.columns = tickers
# заполним пропуски последними известными значениями, так как пропуски чаще всего - это праздничные периоды
# перейдем на недельный интервал и средним значениям на них
df_w = df.fillna(method='ffill').resample('W').ffill()
df_w.head(5)

,EUR,GOLD,Bitcoin,Apple,Exxon,VISA,Oil
Date,,,,,,,
2020-01-05,1.117144,1549.199951,7344.884277,72.735321,58.564049,185.520187,63.049999
2020-01-12,1.111111,1557.500000,8166.554199,75.889969,57.573124,189.600433,59.040001
2020-01-19,1.113958,1558.800049,8929.038086,77.944138,57.090160,200.295197,58.540001
2020-01-26,1.105522,1571.099976,8445.434570,77.841423,55.224911,200.588791,54.189999
2020-02-02,1.102913,1582.900024,9350.529297,75.689430,51.727554,194.688538,51.560001


In [12]:
# % понедельная доходность
df_w_pct = df_w.pct_change().iloc[1:]
df_w_pct.head()

,EUR,GOLD,Bitcoin,Apple,Exxon,VISA,Oil
Date,,,,,,,
2020-01-12,-0.005400,0.005358,0.111870,0.043372,-0.016920,0.021994,-0.063600
2020-01-19,0.002562,0.000835,0.093367,0.027068,-0.008389,0.056407,-0.008469
2020-01-26,-0.007573,0.007891,-0.054161,-0.001318,-0.032672,0.001466,-0.074308
2020-02-02,-0.002360,0.007511,0.107170,-0.027646,-0.063329,-0.029415,-0.048533
2020-02-09,-0.004185,-0.009034,0.047635,0.036443,-0.010464,0.018948,-0.024050


In [15]:
# средняя доходность по активам
mean_returns = df_w_pct.mean()
mean_returns

EUR       -0.000230
GOLD       0.001312
Bitcoin    0.010368
Apple      0.004751
Exxon      0.005587
VISA       0.001632
Oil        0.004626
dtype: float64

In [40]:
# ковариационная матрица
cov_matrix = df_w_pct.cov()
cov_matrix

,EUR,GOLD,Bitcoin,Apple,Exxon,VISA,Oil
EUR,0.000142,0.000103,0.000140,0.000177,0.000103,0.000171,0.000054
GOLD,0.000103,0.000564,0.000447,0.000274,0.000232,0.000199,0.000313
Bitcoin,0.000140,0.000447,0.010059,0.000836,0.001392,0.000668,0.001661
Apple,0.000177,0.000274,0.000836,0.002072,0.000594,0.001202,0.000492
Exxon,0.000103,0.000232,0.001392,0.000594,0.003147,0.000894,0.002551
VISA,0.000171,0.000199,0.000668,0.001202,0.000894,0.001842,0.000652
Oil,0.000054,0.000313,0.001661,0.000492,0.002551,0.000652,0.006160


In [64]:
# получим инициирующие веса простым делением
num_assets = len(tickers)
start_weight = round(1/num_assets, 4)
# start_weight_eq = round(1 - (num_assets - 1) * start_weight,  4) # для уравнивания последнего веса
matrix_weight = np.full(7, start_weight)
matrix_weight

array([0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429])

In [41]:
portfolio_return_start = np.sum(mean_returns * matrix_weight)
portfolio_std_dev_start = np.sqrt(np.dot(matrix_weight.T,np.dot(cov_matrix, matrix_weight)))
sharp_start = portfolio_return_start/portfolio_std_dev_start
sharp_start

0.12505580148512394

In [74]:
# объединим полученные значения в таблицу и выведем ее 
start_result = np.array([portfolio_return_start, portfolio_std_dev_start, sharp_start])
start_result = np.array([portfolio_return_start, portfolio_std_dev_start, sharp_start])
start_result = np.concatenate((start_result, matrix_weight), axis=0)
# СДЕЛАТЬ РАЦИОНАЛЬНЕЕ
start_sim_result = pd.DataFrame(start_result, columns=['Start'], index=['RETURN','stdev','Sharpe', tickers[0], tickers[1], tickers[2], tickers[3], tickers[4], tickers[5], tickers[6]])

print(start_sim_result) 

            Start
RETURN   0.004008
stdev    0.032048
Sharpe   0.125056
EUR      0.142900
GOLD     0.142900
Bitcoin  0.142900
Apple    0.142900
Exxon    0.142900
VISA     0.142900
Oil      0.142900


## СИМУЛЯЦИЯ

In [75]:
#массив из нулей
num_iter = 1000
simulations = np.zeros((4 + len(tickers)- 1, num_iter))

for i in range(num_iter):
        # случайные веса + нормализация, чтоб сумма 1
        weights = np.array(np.random.random(7))
        weights /= np.sum(weights)
        
        # доходность и стандартное отклонение
        portfolio_return = np.sum(mean_returns * weights)
        portfolio_std_dev = np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights)))
        simulations[0,i] = portfolio_return
        simulations[1,i] = portfolio_std_dev
        
        # кэф Шарпа
        simulations[2,i] = simulations[0,i] / simulations[1,i]
        
        # dtcf
        for j in range(len(weights)):
                simulations[j+3,i] = weights[j]

# результат
df_sim = pd.DataFrame(simulations.T, 
                         columns=['RETURN','stdev','Sharpe', tickers[0], tickers[1], tickers[2], tickers[3], tickers[4], tickers[5], tickers[6]])

In [77]:
# максимальный Шарп
max_sharpe = df_sim.iloc[df_sim['Sharpe'].idxmax()]

# минимальное ст. отклонение
min_std = df_sim.iloc[df_sim['stdev'].idxmin()]

print ("Портфель с макс. Шарпом:", max_sharpe, sep='\n')

Портфель с макс. Шарпом:
RETURN     0.004801
stdev      0.033341
Sharpe     0.143998
EUR        0.029989
GOLD       0.239391
Bitcoin    0.166671
Apple      0.272750
Exxon      0.222688
VISA       0.030447
Oil        0.038065
Name: 547, dtype: float64


In [78]:
print ("Портфель с мин. риском:", min_std, sep='\n')

Портфель с мин. риском:
RETURN     0.001217
stdev      0.018136
Sharpe     0.067111
EUR        0.375118
GOLD       0.310509
Bitcoin    0.001461
Apple      0.097064
Exxon      0.011161
VISA       0.196968
Oil        0.007718
Name: 210, dtype: float64
